In [1]:
%load_ext autoreload
%autoreload 2

In [66]:
from reactit.reactit import ReactionGenerator

compounds = {0:'CO2',
              1:'H2O', 
              2:'SO2', 
              3:'NO2', 
              4:'H2S', 
              5:'CH4', 
              6:'CH3', 
              7:'C2H4',
              8:'N2',
              9:'O2',
              10:'H2',
              11:'CO',
              12:'S8',
              13:'H2SO4',
              14:'H2O2',
              15:'HNO3',
              16:'H2SO3',
              17:'HNO2',
              18:'C2H4',
              19:'NH4',
              20:'NH3',
              21:'SO3',
              22:'O3',
              23:'C2H6',
              24:'C3H8',
              25:'C2H5OH',
              26:'HCHO',
              27:'CH3COOH',
              28:'C4H10',
              29:'CH3NH2',
              30:'CH3CONH2'}
compounds = {0:'CO2',
              1:'H2O', 
              2:'SO2', 
              3:'NO2', 
              4:'H2S', 
              5:'N2',
              6:'O2',
              7:'H2',
              8:'CO',
              9:'H2SO4',
              10:'HNO3',
              11:'H2SO3',
              12:'HNO2'}
#compounds = {0:'O2',1:'H2O',2:'H2'}

In [67]:
rg = ReactionGenerator(compounds=compounds)
reactions = rg.iterate(max_length=5)

100%|██████████| 1280/1280 [00:02<00:00, 443.48it/s]


In [68]:
import pandas as pd 
pd.Series(reactions)

0                         2 H2S + 3 O2 = 2 H2SO3
1                            2 NO2 = 1 N2 + 2 O2
2                         1 H2S + 2 O2 = 1 H2SO4
3                       1 O2 + 2 H2SO3 = 2 H2SO4
4                          2 NO2 + 1 H2 = 2 HNO2
                         ...                    
140       1 H2S + 2 HNO2 = 1 N2 + 1 H2 + 1 H2SO4
141       4 H2O + 1 SO2 + 6 NO2 = 1 H2S + 6 HNO3
142      4 H2O + 4 N2 + 5 H2SO4 = 5 H2S + 8 HNO3
143    1 H2O + 1 N2 + 5 H2SO4 = 2 HNO3 + 5 H2SO3
144       4 SO2 + 3 H2 + 2 HNO2 = 1 N2 + 4 H2SO3
Length: 145, dtype: object

In [ ]:
pd.DataFrame(rg.as_dict()).T

AttributeError: 'list' object has no attribute 'as_dict'

In [2]:
def mp_function(iterable):
    reactants = {molecule:mappings.parse_molecule(molecule) for molecule in iterable[0]}
    products = {molecule:mappings.parse_molecule(molecule) for molecule in iterable[1]}
    balanced = mappings.balance_reaction(reactants,products)
    if balanced:
        return(balanced)
    #else:
    #    return(None)
    
import tqdm_pathos 

rlg = ReactionsListGenerator(no_compounds=len(compounds))
reactions = list(rlg.iterate(4))
mappings = MappingtoReaction(reactions = reactions, compounds = compounds)
approved = mappings.remove_indices(reactions)
strings = mappings.convert_to_string(approved)

screened = tqdm_pathos.map(mp_function,strings)
screened = [x for x in screened if x is not None]

  0%|          | 0/12 [00:00<?, ?it/s]

In [3]:
import pandas as pd 
pd.Series(screened)

0             2 H2O = 1 O2 + 2 H2
1             2 NO2 = 1 N2 + 2 O2
2             2 CO2 = 1 O2 + 2 CO
3    2 H2O + 1 H2S = 1 SO2 + 3 H2
4    2 H2O + 2 SO2 = 2 H2S + 3 O2
5     1 SO2 + 1 H2 = 1 H2S + 1 O2
6     1 CO2 + 1 H2 = 1 H2O + 1 CO
7     4 CO2 + 1 N2 = 2 NO2 + 4 CO
8     4 H2O + 1 N2 = 2 NO2 + 4 H2
dtype: object

In [6]:
mappings.as_dict()

AttributeError: 'MappingtoReaction' object has no attribute 'screened'